In [1]:
import pandas as pd
import ast
import os
from dotenv import load_dotenv
load_dotenv()
from pymongo import MongoClient
client = MongoClient(os.getenv("URL"))

In [2]:
df = pd.read_csv("eurocup_2020_results.csv")

In [3]:
def read_d_data(element):
    try:
        return ast.literal_eval(element) 
    except:
        return False

In [4]:
def clean_percentages(num):            
    return int(num.split('%')[0])

In [5]:
def clean_spaces(name):
    return name.strip()

In [6]:
def clean_pen(data):
    if data == 'False':
        return False
    else:
        return int(data)

In [7]:
df["pens_home_score"] = df["pens_home_score"].apply(clean_pen) #Limpio los datos de los penales, ya que el False y el numero de penales era un STR, y al sumarlo 3+4 me daba 34 :'('
df["pens_away_score"] = df["pens_away_score"].apply(clean_pen)

In [8]:
df["team_name_home"]= df["team_name_home"].apply(clean_spaces) #Limpiando espacios en los nombres de los paises.
df["team_name_away"]= df["team_name_away"].apply(clean_spaces)

In [9]:
df["possession_away"] = df["possession_away"].apply(clean_percentages) #Limpiando el porcentaje, y convirtiendo en numero, ya que era un str.
df["possession_home"] = df["possession_home"].apply(clean_percentages)

In [10]:
df["lineup_home"] = df["lineup_home"].apply(read_d_data) #Convierto estos tres STR en listas.
df["lineup_away"] = df["lineup_away"].apply(read_d_data)
df["events_list"] = df["events_list"].apply(read_d_data)

In [11]:
teams = list(df['team_name_home'].unique()) #Creo una lista de paises
teams.sort()

In [12]:
def goals_scored(team):
    goals_home = df[(df['team_name_home'] == team)]['team_home_score'].sum()
    goals_away = df[(df['team_name_away'] == team)]['team_away_score'].sum()
    return int(goals_home + goals_away) 

In [13]:
goals_favor = []
for team in teams:
    goals_favor.append(goals_scored(team))

In [14]:
def goals_against(team):
    against_home = df[(df['team_name_home'] == team)]['team_away_score'].sum()
    against_away = df[(df['team_name_away'] == team)]['team_home_score'].sum()
    return int(against_home + against_away)

In [15]:
goals_received = []
for team in teams:
    goals_received.append(goals_against(team))

In [16]:
def possession_team(team):
    possession_home = df[df['team_name_home'] == team]['possession_home'].sum()
    possession_away = df[df['team_name_away'] == team]['possession_away'].sum()
    matchs=  df[(df['team_name_away'] == team)|(df['team_name_home'] == f'{team}')]['stage'].count()
    possesion_total = (possession_home+possession_away)/matchs
    return round(float(possesion_total), 1)

In [17]:
possession_total = []
for team in teams:
    possession_total.append(possession_team(team))

In [18]:
def shots(team):
    shots_home = df[(df['team_name_home'] == team)]['total_shots_home'].sum()
    shots_away = df[(df['team_name_away'] == team)]['total_shots_away'].sum()
    return int(shots_home + shots_away)

In [19]:
total_shots =[]
for team in teams:
    total_shots.append(shots(team))

In [20]:
def pen(team):
    pen_home = (df[(df['team_name_home'] == team) & (df['pens_home_score'] != False)]['pens_home_score'].sum())
    pen_away = (df[(df['team_name_away'] == team) & (df['pens_away_score'] != False)]['pens_away_score'].sum())
    return (pen_home + pen_away)

In [21]:
penaltys_total = []
for team in teams:
    penaltys_total.append(pen(team))

In [22]:
def add_data(data, index):
    return data[index]

In [23]:
def create_data(index):
    return {
        "team": add_data(teams, index),
        "possession_total": add_data(possession_total, index),
        "goals_favor": add_data(goals_favor, index),
        "goals_received": add_data(goals_received, index),
        "penaltys_total": add_data(penaltys_total, index),
        "shots": add_data(total_shots, index)
    }

In [24]:
data_teams = []
for index, value in enumerate(teams):
    data_teams.append(create_data(index))

In [25]:
data_teams

[{'team': 'Austria',
  'possession_total': 53.5,
  'goals_favor': 5,
  'goals_received': 5,
  'penaltys_total': 0,
  'shots': 53},
 {'team': 'Belgium',
  'possession_total': 53.8,
  'goals_favor': 9,
  'goals_received': 3,
  'penaltys_total': 0,
  'shots': 48},
 {'team': 'Croatia',
  'possession_total': 49.5,
  'goals_favor': 7,
  'goals_received': 8,
  'penaltys_total': 0,
  'shots': 42},
 {'team': 'Czech Republic',
  'possession_total': 48.2,
  'goals_favor': 6,
  'goals_received': 4,
  'penaltys_total': 0,
  'shots': 56},
 {'team': 'Denmark',
  'possession_total': 53.3,
  'goals_favor': 12,
  'goals_received': 7,
  'penaltys_total': 0,
  'shots': 92},
 {'team': 'England',
  'possession_total': 51.0,
  'goals_favor': 11,
  'goals_received': 2,
  'penaltys_total': 2,
  'shots': 63},
 {'team': 'Finland',
  'possession_total': 36.7,
  'goals_favor': 1,
  'goals_received': 3,
  'penaltys_total': 0,
  'shots': 19},
 {'team': 'France',
  'possession_total': 53.2,
  'goals_favor': 7,
  'goa

In [26]:
db = client["euro2020"]

In [27]:
coll_teams = db["data_teams"]

In [28]:
coll_teams.insert_many(data_teams)

In [29]:
#coll_teams.remove() #Para cuando agregue mas datos extraidos.

In [30]:
copa = df["events_list"]

In [31]:
lineup_home_players = df['lineup_home']
lineup_away_players = df['lineup_away']
lineup_total = pd.concat([lineup_home_players, lineup_away_players])

In [32]:
lista_players = set()
for equipo in lineup_total:
    for jugador in equipo:
        lista_players.add(jugador['Player_Name'])

In [33]:
data_players = []
for i in lista_players:
    name = i
    i = {'name':name, 'goals':0, 'assistance':0, 'yellow_cards':0, 'red_cards':0}
    data_players.append(i)

In [36]:
for player in data_players:
    for match in copa:
        for evento in match:
            if (evento['event_type'] == 'Goal' or evento['event_type'] == 'Penalty') and player['name'] in evento['action_player_1']:
                #if 'goals' in player.keys():
                player['goals'] += 1
            if evento['event_type'] == 'Yellow card' and player['name'] in evento['action_player_1']:
                player['yellow_cards'] +=1
            if evento['event_type'] == 'Red card' and player['name'] in evento['action_player_1']:
                player['red_cards'] +=1
            if evento['event_type'] == 'Goal' and 'action_player_2' in evento.keys():
                if player['name'] in evento['action_player_2']:
                    player['assistance'] += 1

In [37]:
coll_players = db["data_players"]

In [38]:
coll_players.insert_many(data_players)